In [20]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
pd.options.display.max_columns = None
import joblib
from itertools import chain
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif

In [2]:
df = pd.read_csv('data/data.csv', low_memory=False)

First we fix the encoding to transform codes, such as 8 and 9 to NaN if they represent missing values. Also, we drop several columns that we found not present in the data dictionary, since we do not know what do they represent.

Then we proceed with removing columns that contain > 20% of missing values.

In [3]:
max_vals = df.select_dtypes(include='number').max()
max_9 = max_vals[max_vals == 9].index.tolist()
for col in max_9:
    df.loc[df[col] == 9, col] = np.nan 
max_vals = df.select_dtypes(include='number').max()
max_8 = max_vals[max_vals == 8].index.tolist()

missing_subset = {'ALCFREQ', 'HATTMULT', 'STROKMUL', 'TIAMULT', 'ARTHTYPE', 'ARTHUPEX', 'ARTHLOEX', 'ARTHSPIN', 'ARTHUNK', 'CVDCOG', 'STROKCOG', 'CVDIMAG', 'CVDIMAG1', 'CVDIMAG2', 'CVDIMAG3', 'CVDIMAG4', 'PDNORMAL', 'SPEECH', 'FACEXP', 'TRESTRHD', 'TRESTLHD', 'TRESTRFT', 'TRESTLFT', 'TRACTRHD', 'TRACTLHD', 'RIGDNECK', 'RIGDUPRT', 'RIGDUPLF', 'RIGDLORT', 'RIGDLOLF', 'TAPSRT', 'TAPSLF', 'HANDMOVR', 'HANDMOVL', 'HANDALTR', 'HANDALTL', 'LEGRT', 'LEGLF', 'ARISING', 'POSTURE', 'GAIT', 'POSSTAB', 'BRADYKIN', 'RESTTRL', 'RESTTRR', 'SLOWINGL', 'SLOWINGR', 'RIGIDL', 'RIGIDR', 'BRADY', 'POSTINST', 'CORTDEF', 'SIVDFIND', 'CVDMOTL', 'CVDMOTR', 'CORTVISL', 'CORTVISR', 'SOMATL', 'SOMATR', 'EYEPSP', 'DYSPSP', 'AXIALPSP', 'GAITPSP', 'APRAXSP', 'APRAXL', 'APRAXR', 'CORTSENL', 'CORTSENR', 'ATAXL', 'ATAXR', 'ALIENLML', 'ALIENLMR', 'DYSTONL', 'DYSTONR', 'MYOCLLT', 'MYOCLRT', 'MOMOPARK', 'MOMOALS', 'AMNDEM', 'PCA', 'NAMNDEM', 'AMYLPET', 'AMYLCSF', 'FDGAD', 'HIPPATR', 'TAUPETAD', 'CSFTAU', 'FDGFTLD', 'TPETFTLD', 'MRFTLD', 'DATSCAN', 'IMAGLINF', 'IMAGLAC', 'IMAGMACH', 'IMAGMICH', 'IMAGMWMH', 'IMAGEWMH', 'CANCER', 'MYOINF', 'CONGHRT', 'AFIBRILL', 'HYPERT', 'ANGINA', 'HYPCHOL', 'VB12DEF', 'THYDIS', 'ARTH', 'ARTYPE', 'ARTUPEX', 'ARTLOEX', 'ARTSPIN', 'ARTUNKN', 'URINEINC', 'BOWLINC', 'SLEEPAP', 'REMDIS', 'HYPOSOM', 'SLEEPOTH', 'ANGIOCP', 'ANGIOPCI', 'PACEMAKE', 'HVALVE', 'ANTIENC'}
cols_to_change = list(missing_subset.intersection(max_8))
df[cols_to_change] = df[cols_to_change].replace(8, np.nan)
max_vals = df.select_dtypes(include='number').max()
max_8 = max_vals[max_vals == 8].index.tolist()

df = df.drop(columns=['NPWBRF', 'NACCBRNN', 'NPGRCCA', 'NPGRLA', 'NPGRHA', 'NPGRSNH', 'NPGRLCH', 'NACCAVAS', 'NPTAN', 'NPABAN', 'NPASAN', 'NPTDPAN', 'NPTHAL', 'NACCBRAA', 'NACCNEUR', 'NPADNC', 'NACCDIFF', 'NACCAMY', 'NPINF', 'NACCINF', 'NPHEMO', 'NPHEMO1', 'NPHEMO2', 'NPHEMO3', 'NPOLD', 'NPOLD1', 'NPOLD2', 'NPOLD3', 'NPOLD4', 'NACCMICR', 'NPOLDD', 'NPOLDD1', 'NPOLDD2', 'NPOLDD3', 'NPOLDD4', 'NACCHEM', 'NACCARTE', 'NPWMR', 'NPPATH', 'NACCNEC', 'NPPATH2', 'NPPATH3', 'NPPATH4', 'NPPATH5', 'NPPATH6', 'NPPATH7', 'NPPATH8', 'NPPATH9', 'NPPATH10', 'NPPATH11', 'NACCLEWY', 'NPLBOD', 'NPNLOSS', 'NPHIPSCL', 'NPFTDTAU', 'NACCPICK', 'NPFTDT2', 'NACCCBD', 'NACCPROG', 'NPFTDT5', 'NPFTDT6', 'NPFTDT7', 'NPFTDT8', 'NPFTDT9', 'NPFTDT10', 'NPFTDTDP', 'NPALSMND', 'NPOFTD', 'NPOFTD1', 'NPOFTD2', 'NPOFTD3', 'NPOFTD4', 'NPOFTD5', 'NPTDPA', 'NPTDPB', 'NPTDPC', 'NPTDPD', 'NPTDPE', 'NPPDXA', 'NPPDXB', 'NACCPRIO', 'NPPDXD', 'NPPDXE', 'NPPDXF', 'NPPDXG', 'NPPDXH', 'NPPDXI', 'NPPDXJ', 'NPPDXK', 'NPPDXL', 'NPPDXM', 'NPPDXN', 'NPPDXP', 'NPPDXQ', 'NPARTAG', 'NPATGSEV', 'NPATGAMY', 'NPATGAM1', 'NPATGAM2', 'NPATGAM3', 'NPATGAM4', 'NPATGAM5', 'NPATGFRN', 'NPATGFR1', 'NPATGFR2', 'NPATGFR3', 'NPATGFR4'])

initial = df.shape[1]
threshold = 0.2 * len(df)
df = df.dropna(thresh=threshold, axis=1)
remaining = df.shape[1]
dropped = initial - remaining

print(f"initial: {initial}")
print(f"remaining: {remaining}")
print(f"dropped: {dropped}")

initial: 862
remaining: 527
dropped: 335


Now we apply the transformation mentioned during EDA - coalescing columns BILLS, SHOPPING, STOVE, TRAVEL into one column to reduce dimensionality
At the end we also create a deep copy of the dataframe, for performance reasons, to aid with fragmentation

In [4]:
impairment_vars = ['BILLS', 'SHOPPING', 'STOVE', 'TRAVEL']

functional_impairment = df[impairment_vars].sum(axis=1, skipna=True)

df = pd.concat([df, functional_impairment.rename('FUNCTIONAL_IMPAIRMENT')], axis=1)
df.drop(columns=impairment_vars, inplace=True)

df = df.copy()

We follow by dropping the target variable - OUTCOME_EVENTMCI as well as TIME, since it is not a predictor
Then we proceed by creating a training and testing datasets, split 80/20

In [5]:
X = df.drop(columns=['OUTCOME_EVENTMCI', 'TIME'])
y = df['OUTCOME_EVENTMCI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22, stratify=y)

Now we can start building the preprocessing pipeline

> First we create a transformer that is going to handle outliers. We will be using the transformer in the pipeline.

In [6]:
class HandleOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.3, upper_quantile=0.7):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile

    def fit(self, X, y=None):
        X = pd.DataFrame(X)
        self.quantile_bounds_ = {}
        numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
        
        for col in numeric_columns:
            Q1 = X[col].quantile(self.lower_quantile)
            Q2 = X[col].quantile(self.upper_quantile)
            IQR = Q2 - Q1
            self.quantile_bounds_[col] = {
                'lower_bound': Q1 - 1.5 * IQR,
                'upper_bound': Q2 + 1.5 * IQR
            }
        return self

    def transform(self, X):
        X = pd.DataFrame(X).copy()
        numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns

        for col in numeric_columns:
            if col not in self.quantile_bounds_:
                continue  
            bounds = self.quantile_bounds_[col]
            mean_value = X[col].mean()
            
            X[col] = np.where(X[col] < bounds['lower_bound'], mean_value, 
                              np.where(X[col] > bounds['upper_bound'], mean_value, X[col]))
        return X.values

> Now we can split the dataset to numerical and categorical columns and create two sub-pipelines and process them separately, in the end we will join them again

> The numerical pipeline include a simple imputer using median, clipping outliers in the 0.3 and 0.7 quantiles and scaling the remaining values using standardscaler

> The categorical pipeline is being imputed using most_frequent method, and the values are then encoded using one hot encoder.


In [17]:
num_cols = X.select_dtypes(['number']).columns
cat_cols = X.select_dtypes(['object']).columns

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('outlier', HandleOutliers(lower_quantile=0.3, upper_quantile=0.7)),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

var_thresh_low = VarianceThreshold(threshold=0.01)
var_thresh_mid = VarianceThreshold(threshold=0.05)
var_thresh_high = VarianceThreshold(threshold=0.1)

preprocessor.fit(X_train)
vt_low_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('variance_threshold', var_thresh_low)
])
vt_mid_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('variance_threshold', var_thresh_mid)
])
vt_high_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('variance_threshold', var_thresh_high)
])

X_vt_low = vt_low_pipeline.fit_transform(X_train)
X_vt_mid = vt_mid_pipeline.fit_transform(X_train)
X_vt_high = vt_high_pipeline.fit_transform(X_train)

print("Original X_train shape:", X_train.shape)
print("X_train shape after variance threshold of 0.01:", X_vt_low.shape)
print("X_train shape after variance threshold of 0.05:", X_vt_mid.shape)
print("X_train shape after variance threshold of 0.1:", X_vt_high.shape)

Original X_train shape: (23738, 522)
X_train shape after variance threshold of 0.01: (23738, 589)
X_train shape after variance threshold of 0.05: (23738, 522)
X_train shape after variance threshold of 0.1: (23738, 520)


In [21]:
num_cols = X.select_dtypes(['number']).columns
cat_cols = X.select_dtypes(['object']).columns

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('outlier', HandleOutliers(lower_quantile=0.3, upper_quantile=0.7)),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

var_thresh = VarianceThreshold(threshold=0.1)
score_funcs = [f_classif, mutual_info_classif]  
n_features_rfe = [20, 10]  
rfe_estimators = [LogisticRegression(max_iter=1000, solver='liblinear'), RandomForestClassifier()]  

results = []

for n_features in n_features_rfe:
    k = 50 if n_features == 20 else 20

    for score_func in score_funcs:
        for estimator in rfe_estimators:
            select_k = SelectKBest(score_func=score_func, k=k)
            rfe = RFE(estimator=estimator, n_features_to_select=n_features)
            full_pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('variance_threshold', var_thresh),
                ('select_k_best', select_k),
                ('rfe', rfe),
                ('classifier', LogisticRegression(max_iter=1000, solver='liblinear'))
            ])

            full_pipeline.fit(X_train, y_train)

            train_score = full_pipeline.score(X_train, y_train)
            test_predictions = full_pipeline.predict(X_test)
            test_accuracy = accuracy_score(y_test, test_predictions)

            results.append({
                'k': k,
                'score_func': score_func.__name__,
                'n_features_rfe': n_features,
                'rfe_estimator': estimator.__class__.__name__,
                'train_score': train_score,
                'test_accuracy': test_accuracy,
            })
results_df = pd.DataFrame(results)
print(results_df)

    k           score_func  n_features_rfe           rfe_estimator  \
0  50            f_classif              20      LogisticRegression   
1  50            f_classif              20  RandomForestClassifier   
2  50  mutual_info_classif              20      LogisticRegression   
3  50  mutual_info_classif              20  RandomForestClassifier   
4  20            f_classif              10      LogisticRegression   
5  20            f_classif              10  RandomForestClassifier   
6  20  mutual_info_classif              10      LogisticRegression   
7  20  mutual_info_classif              10  RandomForestClassifier   

   train_score  test_accuracy  
0     0.974724       0.973547  
1     0.974556       0.974221  
2     0.974724       0.973547  
3     0.974556       0.974221  
4     0.931671       0.935973  
5     0.931671       0.935973  
6     0.931671       0.935973  
7     0.931586       0.935973  


In [22]:
num_cols = X.select_dtypes(['number']).columns
cat_cols = X.select_dtypes(['object']).columns

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('outlier', HandleOutliers(lower_quantile=0.3, upper_quantile=0.7)),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

var_thresh = VarianceThreshold(threshold=0.1)
select_k = SelectKBest(score_func=mutual_info_classif, k=50)
log_reg = LogisticRegression(max_iter=1000, solver='liblinear')
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=20)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('variance_threshold', var_thresh),
    ('select_k_best', select_k),
    ('rfe', rfe),
    ('classifier', LogisticRegression(max_iter=200))
])

full_pipeline.fit(X_train, y_train)

train_score = full_pipeline.score(X_train, y_train)
test_predictions = full_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

pipeline_filename = 'pipeline.pkl'
joblib.dump(full_pipeline, pipeline_filename)

train_score, test_accuracy, pipeline_filename

(0.9744291852725587, 0.9742207245155855, 'pipeline.pkl')